In [1]:
import numpy as np
import pandas as pd 
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from category_encoders.count import CountEncoder
import gc

NUM_FOLDS=5
RANDOM_STATE = 800


#     ░░░░░░░░░░░░        ▒▒▓▓██▓▓▓▓▓▓▓▓▓▓██▓▓▓▓▓▓    ░░░░░░░░░░░░░░        
#           ░░░░░░░░░░░░  ██████░░░░░░░░░░░░░░░░  ░░░░░░████░░░░░░░░░░░░░░        
#           ░░░░░░░░░░████  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░████░░░░░░░░          
#                   ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ██                
#               ░░██    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██              
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#               ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██            
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  ░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#       ▓▓░░░░░░████░░▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░░░░░░░▒▒▓▓    
#       ██░░▒▒▓▓▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██▒▒░░██▒▒▒▒▒▒▒▒▒▒██░░░░░░░░░░░░░░▓▓░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░▒▒██▒▒▒▒▒▒██░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░░░░░██████▒▒░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░██░░░░░░░░░░██    
#       ██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░▒▒▓▓░░██░░░░░░░░░░██    
#       ██░░░░░░░░▒▒░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░██░░░░██░░░░░░░░░░██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▓▓▓▓▓▓▓▓░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓██░░░░██▓▓▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▓▓▒▒▒▒██▓▓░░░░░░▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒████▒▒▒▒████████████░░░░░░░░░░░░░░░░░░░░████████████▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██  ██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▓▓  ░░██░░░░░░░░░░░░░░░░░░░░██░░░░██▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒████░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▓▓██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██████░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░██▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██▒▒▒▒██████░░░░░░░░░░░░░░░░░░░░██████▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████▒▒▒▒▒▒▒▒██░░░░████████████████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒▒▒██░░░░░░██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████░░░░░░██████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓░░    
#       ░░██▒▒▒▒██▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓██░░░░░░██▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    


In [2]:
def negative_downsample(df, target, neg_frac = .1):
    # Negative downsampling
    df_pos = df[df[target]==1]
    df_neg = df[df[target]==0]
    
    df_neg = df_neg.sample(int(df.shape[0] * neg_frac), random_state=42)
    return pd.concat([df_pos,df_neg]).sort_index()

In [3]:
dtypes = {'label': 'int8',
 'uid': 'int32',
 'task_id': 'int16',
 'adv_id': 'int16',
 'creat_type_cd': 'int8',
 'adv_prim_id': 'int16',
 'dev_id': 'int8',
 'inter_type_cd': 'int8',
 'slot_id': 'int8',
 'spread_app_id': 'int8',
 'tags': 'int8',
 'app_first_class': 'int8',
 'app_second_class': 'int8',
 'age': 'int8',
 'city': 'int16',
 'city_rank': 'int8',
 'device_name': 'int8',
 'device_size': 'int16',
 'career': 'int8',
 'gender': 'int8',
 'net_type': 'int8',
 'residence': 'int8',
 'his_app_size': 'int8',
 'his_on_shelf_time': 'int8',
 'app_score': 'int8',
 'emui_dev': 'int8',
 'list_time': 'int8',
 'device_price': 'int8',
 'up_life_duration': 'int8',
 'up_membership_grade': 'int8',
 'membership_life_duration': 'int8',
 'consume_purchase': 'int8',
 'communication_onlinerate': 'object',
 'communication_avgonline_30d': 'int8',
 'indu_name': 'int8',
 'pt_d': 'int8'}

In [8]:
train_df  = pd.read_csv('train_data/train_data.csv', sep="|",dtype = dtypes ).reset_index()
test_df = pd.read_csv('test_data_A.csv', sep="|",dtype = dtypes ).reset_index()



b = pd.concat([train_df.drop('label',axis=1),test_df.drop('id',axis=1)]).sort_values(['pt_d','index']).reset_index()
for col in ['task_id','adv_prim_id']:
    conts = []
    for i in range(2,9):
        a = b[b.pt_d==i-1].groupby(['uid',col]).size().reset_index().rename({0:f'uid_{col}_ptd'},axis=1)
        a['pt_d'] = i

        conts.append(a)
    c = pd.concat(conts)
    train_df = pd.merge(train_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
    test_df = pd.merge(test_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
    gc.collect()

for col in ['task_id','adv_id','slot_id','spread_app_id']:
    conts = []
    for i in range(2,9):
        a = b[b.pt_d.isin(range(i))].groupby(['uid',col]).size().reset_index().rename({0:f'uid_{col}_ptd_rng'},axis=1)
        a['pt_d'] = i

        conts.append(a)
    c = pd.concat(conts)
    train_df = pd.merge(train_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
    test_df = pd.merge(test_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
    gc.collect()

b = train_df.sort_values(['pt_d','index']).reset_index()


for col in ['uid','task_id','adv_id']:
    conts = []
    for i in range(2,9):
        a =  b[b.pt_d==i-1][[col,'label']].groupby([col]).mean().reset_index().rename({'label':f'{col}_targ_mean'},axis=1)
        a['pt_d'] = i

        conts.append(a)
    c = pd.concat(conts)
    train_df = pd.merge(train_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    test_df = pd.merge(test_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    gc.collect()




for col in ['uid','task_id']:
    conts = []
    for i in range(2,9):
        a =  b[b.pt_d.isin(range(i))][[col,'label']].groupby([col]).mean().reset_index().rename({'label':f'{col}_targ_mean_sumed'},axis=1)
        a['pt_d'] = i

        conts.append(a)
    c = pd.concat(conts)
    train_df = pd.merge(train_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    test_df = pd.merge(test_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    
    gc.collect()









cnt = CountEncoder(cols=['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','slot_id','spread_app_id','indu_name'])

cnt.fit(pd.concat([train_df.drop('label',axis =1),test_df.drop('id',axis=1)]))

train_m = cnt.transform(train_df.drop('label',axis=1))
test_m = cnt.transform(test_df.drop('id',axis=1))

In [9]:
train_m['label'] = train_df.label

In [14]:
FEATS = ['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','slot_id','spread_app_id','indu_name','age','uid_task_id_ptd','uid_adv_prim_id_ptd',
        'uid_task_id_ptd','uid_adv_id_ptd_rng','uid_slot_id_ptd_rng','uid_spread_app_id_ptd_rng','his_app_size','app_first_class','gender',
        'career','device_name','uid_targ_mean','task_id_targ_mean','adv_id_targ_mean','uid_targ_mean_sumed',
        'task_id_targ_mean_sumed']

PARAMS = {  'learning_rate':0.05, 'max_bin': 100, 'max_depth': -1, 'min_child_samples': 5000,
        'n_estimators': 200,
        'num_leaves': 250,'objective': 'binary', 'n_jobs': -1,'subsample':.7,'boost_from_average': False}


In [15]:
train_m[:6277359].pt_d.value_counts()

1    6277359
Name: pt_d, dtype: int64

In [16]:
train_m[35897957:].pt_d.value_counts()

7    6009176
Name: pt_d, dtype: int64

In [17]:
train_downded = negative_downsample(train_m[6277359:35897957],'label')
train_data = lgb.Dataset(train_downded.drop('label',axis=1)[FEATS], label=train_downded.label, free_raw_data=True)
aa = lgb.train(PARAMS,train_data)
preds = aa.predict(train_m[35897957:].drop('label',axis=1)[FEATS])
roc_auc_score(train_m[35897957:].label,preds)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.8296922255901467

In [20]:
train_downded = negative_downsample(train_m[6277359:],'label')
train_data = lgb.Dataset(train_downded.drop('label',axis=1)[FEATS], label=train_downded.label, free_raw_data=True)
aa = lgb.train(PARAMS,train_data)
preds = aa.predict(test_m[FEATS])


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [21]:
test_df['probability']= preds

In [22]:
test_df[['id','probability']].set_index('id').to_csv('0.8296922255901467.csv')